## Building a Convolutional Neural Network with PyTorch
#### Model A:
* 2 Convolutional Layers: Same Padding (same output size)
* 2 Max Pooling Layers
* 1 Fully Connected Layer

#### Steps:
* Step 1: Load Dataset
* Step 2: Make Dataset Iterable
* Step 3: Create Model Class
* Step 4: Instantiate Model Class
* Step 5: Instantiate Loss Class
* Step 6: Instantiate Optimizer Class
* Step 7: Train Model

#### Step 1:

In [2]:
#Step 1: 
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets


In [4]:
#MNIST Dataset
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

In [5]:
#Size of Training Dataset (Excluding Labels)
print(train_dataset.train_data.size())

torch.Size([60000, 28, 28])


C:\Users\salee\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:64: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [7]:
#Size of our training dataset labels
print(train_dataset.train_labels.size())

torch.Size([60000])


C:\Users\salee\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:54: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


In [9]:
#Size of our testing dataset labels
print(test_dataset.test_labels.size())

torch.Size([10000])


C:\Users\salee\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:59: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


#### Step 2: Make Dataset Iterable

In [10]:
#Load Dataset into Dataloader
batch_size = 100
n_iters = 3000
num_epochs = 2
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

#### Step 3: Create Model Class

In [12]:
#Define our simple 2 convolutional layer CNN
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()

        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()

        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(32 * 7 * 7, 10) 

    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)

        # Max pool 1
        out = self.maxpool1(out)

        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)

        # Max pool 2 
        out = self.maxpool2(out)

        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)

        return out

#### Step 4: Instantiate Model Class

In [13]:
#Our model
model = CNNModel()

#### Step 5: Instantiate Loss Class
* Convolutional Neural Network: Cross Entropy Loss
    * Feedforward Neural Network: Cross Entropy Loss
    * Logistic Regression: Cross Entropy Loss
    * Linear Regression: MSE

In [14]:
# Our cross entropy loss
criterion = nn.CrossEntropyLoss()

#### Step 6: Instantiate Optimizer Class

In [15]:
# Optimizer
learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [16]:
# PARAMETERS IN-DEPTH
# Print model's parameter
print(model.parameters())

print(len(list(model.parameters())))

# Convolution 1: 16 Kernels
print(list(model.parameters())[0].size())

# Convolution 1 Bias: 16 Kernels
print(list(model.parameters())[1].size())

# Convolution 2: 32 Kernels with depth = 16
print(list(model.parameters())[2].size())

# Convolution 2 Bias: 32 Kernels with depth = 16
print(list(model.parameters())[3].size())

# Fully Connected Layer 1
print(list(model.parameters())[4].size())

# Fully Connected Layer Bias
print(list(model.parameters())[5].size())

<generator object Module.parameters at 0x0000019B70A31EB0>
6
torch.Size([16, 1, 5, 5])
torch.Size([16])
torch.Size([32, 16, 5, 5])
torch.Size([32])
torch.Size([10, 1568])
torch.Size([10])


#### Step 7: Train Model
* Process:
    * Convert inputs to tensors with gradient accumulation abilities
        * CNN Input: (1, 28, 28)
        * Feedforward NN Input: (1, 28*28)
    * Clear gradient buffets
    * Get output given inputs
    * Get loss
    * Get gradients w.r.t. parameters
    * Update parameters using gradients
        * parameters = parameters - learning_rate * parameters_gradients
    * REPEAT

In [17]:
# Model training
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images
        images = images.requires_grad_()

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images
                images = images.requires_grad_()

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 0.30295053124427795. Accuracy: 91.7300033569336


KeyboardInterrupt: 